# 🏭 AI-Powered Manufacturing Intelligence Demo w/ Snowflake Cortex

## 🎯 Purpose
This notebook demonstrates how manufacturing teams leverage Snowflake's Cortex AI capabilities to transform traditional JIRA production data into actionable manufacturing intelligence. Experience conversational AI that answers complex operational questions, identifies critical failure patterns, and generates predictive maintenance insights.

## 💡 Why do this?
- 🔍 **Root Cause Analysis**: AI-driven pattern recognition to identify recurring problems and their sources
- ⚡ **Downtime Prevention**: Predictive insights to prevent equipment failures before they occur
- 🔧 **Maintenance Optimization**: Intelligent scheduling based on failure patterns and asset health
- 📊 **Compliance Monitoring**: Automated tracking of safety and quality requirements
- 🏆 **Process Excellence**: Identify best practices and replicate high-performing operations
- 📈 **Operational Efficiency**: Real-time insights into production line performance and bottlenecks

## 🛠️ Solution Components
Note: This demo uses SQL & Python to showcase Snowflake Cortex capabilities for manufacturing intelligence

1. 📊 **Manufacturing Operations Dashboard**
   - Interactive dashboards showing equipment health and incident trends
   - Dynamic filtering by production line, shift, and equipment type

2. 🤖 **AI-Powered Manufacturing Analysis**
   - Equipment summarization using `AI_AGG` across incident history
   - Automated root cause detection with `AI_FILTER`
   - Multi-dimensional failure analysis with `ENTITY_SENTIMENT`
   - Maintenance effectiveness scoring and asset assessment

3. ⚙️ **Automated Manufacturing Intelligence Pipeline**
   - Real-time processing of new production incidents
   - Automatic equipment health scoring updates
   - Failure pattern tracking and predictive alerts

4. 🎯 **Conversational Manufacturing Assistant**
   - Natural language queries for maintenance planning
   - Semantic search across incident history and equipment data
   - Contextual recommendations and maintenance actions

## 🌟 Common Scenarios
*"What's causing the most downtime on Production Line A this month?"*

*"Which equipment showed repeated failures and what was the root cause?"*

*"Show me all compliance incidents that took longer than 24 hours to resolve."*

*"Generate a maintenance plan for Equipment Unit 101 based on recent failure patterns."*


In [ ]:
# Core packages for manufacturing analytics and visualization
import streamlit as st
import pandas as pd
import altair as alt
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go

# Snowflake session for AI-powered analytics
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
# JIRA data configuration - update these if using your own production data or loaded your data somewhere else!
issues_table = 'AI_SOL.JIRA.ISSUES'
projects_table = 'AI_SOL.JIRA.PROJECTS'
users_table = 'AI_SOL.JIRA.USERS'


# Display table locations for reference
st.markdown(f"""
**JIRA Data Sources:**
- 🎫 JIRA Issues: `{issues_table}`
- 🏭 Projects: `{projects_table}`  
- 🔧 Users: `{users_table}`
""")


# 🔍 AI-Powered Equipment Intelligence

Now let's dive into the real power of Cortex AI for manufacturing intelligence. We'll use `AI_AGG` to automatically summarize equipment performance and identify key insights from production incidents.

This is where traditional JIRA reporting transforms into intelligent manufacturing assistance.


In [ ]:
-- AI-powered JIRA issue analysis
-- This demonstrates how we can instantly analyze JIRA issues
-- by summarizing patterns and extracting key insights

SELECT 
    i.PROJECT_NAME,
    i.PROJECT_CODE,
    i.ASSIGNEE,
    COUNT(i.ID) as total_issues,
    COUNT(CASE WHEN i.RESOLVED_TS IS NOT NULL THEN 1 END) as resolved_issues,
    AI_AGG(
        i.SUMMARY,
        'Summarize the project issues and patterns. Highlight recurring problems, common themes, resolution patterns, and current status. Include any patterns related to assignees, priorities, or issue types.'
    ) as project_analysis
FROM {{issues_table}} i
WHERE i.PROJECT_NAME = 'Manufacturing Operations' AND ASSIGNEE='James Müller'
GROUP BY i.PROJECT_NAME, i.PROJECT_CODE, i.ASSIGNEE
LIMIT 30;


In [ ]:
# You can reference results from other cells - handy when you'd like to display the output :)
st.write(AI_AGG.to_pandas()["PROJECT_ANALYSIS"][0])

In [ ]:
# JIRA Project Analysis Assistant
# although we call it AISQL - you can call the same powerful functions with Python
# let's wrap AI_AGG into an application that let's us monitor various projects and people
from snowflake.snowpark.functions import col, count_distinct, max, ai_agg

st.header("📊 JIRA Project Analysis Assistant")

# Create DataFrames from tables
issues_df = session.table(issues_table)
projects_df = session.table(projects_table)
users_df = session.table(users_table)

# Create interactive project analysis tool
with st.form("project_analysis"):
    st.markdown("### Generate project insights with AI analysis")
    
    # Get list of projects for selection 
    project_list = (issues_df.select('PROJECT_NAME').distinct().sort('PROJECT_NAME').to_pandas()['PROJECT_NAME'].tolist())
    
    selected_project = st.selectbox(
        "Select Project for Analysis:",
        options=project_list,
        index=0 if project_list else None,
        key="project_select"
    )
    
    # Get assignees for selected project
    if selected_project:
        assignee_list = (issues_df
            .filter(col('PROJECT_NAME') == selected_project)
            .select('ASSIGNEE')
            .distinct()
            .sort('ASSIGNEE')
            .to_pandas()['ASSIGNEE']
            .tolist()
        )
        
        selected_assignee = st.selectbox(
            "Select Assignee (Optional):",
            options=["All Assignees"] + assignee_list,
            key="assignee_select"
        )
    
    analysis_submitted = st.form_submit_button("Generate Project Analysis")

if analysis_submitted and selected_project:
    # Build filter conditions
    filter_conditions = [col('PROJECT_NAME') == selected_project]
    if selected_assignee and selected_assignee != "All Assignees":
        filter_conditions.append(col('ASSIGNEE') == selected_assignee)
    
    # Get comprehensive project intelligence
    project_analysis = (issues_df
        .filter(*filter_conditions)
        .group_by(col('PROJECT_NAME'), col('PROJECT_CODE'))
        .agg(
            count_distinct('ID').alias('TOTAL_ISSUES'),
            count_distinct(col('ASSIGNEE')).alias('UNIQUE_ASSIGNEES'),
            max('CREATED').alias('LAST_ISSUE_DATE'),
            ai_agg(
                col('SUMMARY'),
                'Create a comprehensive project analysis. Include: 1) Common issue themes and patterns 2) Project health assessment 3) Team workload analysis 4) Priority distribution 5) Recommendations for improvement 6) Key areas of focus. Format as project brief.'
            ).alias('PROJECT_ANALYSIS')
        )
        .collect()
    )
    
    if project_analysis:
        analysis = project_analysis[0]
        
        st.success(f"Project Analysis Generated for {selected_project}" + 
                  (f" - {selected_assignee}" if selected_assignee != "All Assignees" else ""))
        
        # Display key metrics
        col1, col2, col3, col4 = st.columns(4)
        with col1:
            st.metric("Project Code", analysis['PROJECT_CODE'])
        with col2:
            st.metric("Total Issues", analysis['TOTAL_ISSUES'])
        with col3:
            st.metric("Team Members", analysis['UNIQUE_ASSIGNEES'])
        with col4:
            last_issue = analysis['LAST_ISSUE_DATE']
            if last_issue:
                st.metric("Last Issue", last_issue.strftime('%Y-%m-%d'))
            else:
                st.metric("Last Issue", "N/A")
        
        # Display AI-generated analysis
        st.markdown("### 📋 AI-Generated Project Analysis")
        st.markdown(analysis['PROJECT_ANALYSIS'])
        
        # Get recent issues for this project
        recent_issues = (issues_df
            .filter(*filter_conditions)
            .select(
                col('KEY'),
                col('SUMMARY'),
                col('PRIORITY'),
                col('STATUS'),
                col('ASSIGNEE'),
                col('CREATED')
            )
            .order_by(col('CREATED').desc())
            .limit(10)
            .to_pandas()
        )
        
        if not recent_issues.empty:
            st.markdown("### 🎫 Recent Issues")
            st.dataframe(recent_issues, use_container_width=True)


# 🔍 Root Cause Analysis with AI_CLASSIFY

One of the key challenges in manufacturing is identifying root causes scattered across incident reports. `AI_CLASSIFY` allows us to intelligently identify recurring problems and their sources, even when root causes aren't explicitly documented.

This goes beyond simple keyword matching - it understands context and patterns in failure descriptions.


In [ ]:
-- Intelligent issue classification across all JIRA issues
-- This finds patterns and classifies issues by type and urgency

SELECT 
    i.KEY,
    i.SUMMARY,
    AI_CLASSIFY(
        i.SUMMARY,
        [
            {'label': 'Bearing System Failure', 'description': 'bearing overheating, defective bearings, or bearing-induced alignment drift'},
            {'label': 'Electrical Component Failure', 'description': 'sensor failures, electrical faults in valves or sensors, or electrical system malfunctions'},
            {'label': 'Hydraulic System Leak', 'description': 'leaks in valves, nozzles, seals, or other hydraulic components'},
            {'label': 'Sensor Calibration Issue', 'description': 'sensor misread failures, accuracy problems, or calibration drift'},
            {'label': 'Mechanical Alignment Problem', 'description': 'alignment drift in nozzles, bearings, sensors, or other mechanical components'},
            {'label': 'Thermal Management Failure', 'description': 'overheating of bearings, valves, or other components due to thermal issues'},
            {'label': 'Regulatory Compliance Violation', 'description': 'compliance deviations, regulatory reporting issues, or procedural violations'},
            {'label': 'Recurring Incident Escalation', 'description': 'repeated failures, unresolved root causes, or escalated production disruptions'}
        ],
        {
            'task_description': 'Classify manufacturing issues by detailed category for operational analysis',
            'output_mode': 'single' -- this can be 'multi' if appropriate!
        }
    ) as issue_category,
    i.STATUS
FROM {{issues_table}} i
ORDER BY i.CREATED DESC;

In [ ]:
# Get data from cell9 and process for visualization
df = AI_CLASSIFY.to_pandas()

# Extract labels from JSON strings for both columns
df['issue_category'] = df['ISSUE_CATEGORY'].str.extract(r'"labels":\s*\[\s*"([^"]+)"\s*\]')

# Display summary metrics
st.markdown("### Summary Statistics")

most_common_category = df['issue_category'].mode()[0]
category_count = df['issue_category'].value_counts().iloc[0]
st.metric("Most Common Issue Category", 
          most_common_category,
          f"{category_count} occurrences")

    
# Create category chart
category_chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('count():Q', title='Count'),
    y=alt.Y('issue_category:N', title='Issue Category', sort='-x'),
    color=alt.Color('issue_category:N', legend=None, scale=alt.Scale(scheme='blues')),
    tooltip=['issue_category', 'count()']
).properties(height=300)

st.altair_chart(category_chart, use_container_width=True)

# ⚡ Maintenance Effectiveness Analysis with AI Intelligence

Maintenance effectiveness is critical for manufacturing operations. Here we use AI to automatically analyze maintenance patterns, resolution times, and repeat failures to assess the health of our maintenance processes.

This transforms subjective maintenance assessment into data-driven insights for continuous improvement.

In this case using `AI_COMPLETE`, which allows us to run a custom prompt with our model of choice, we can define a strict schema for the LLM to follow. This is perfect for engineering tasks, where structure ensures consistency and pipeline reliability. [See more](https://docs.snowflake.com/en/user-guide/snowflake-cortex/complete-structured-outputs)


In [ ]:
-- With structured outputs, we can extract multiple entities in one request.
-- This reduces multiple round hops to the same model for information.
-- For accuracy, make the requests related by topic where possible

-- Looking to run this on change data periodically? 
-- Using a dynamic table means we can define the structure
-- and Snowflake will manage the pipeline!

-- CREATE DYNAMIC TABLE LLM_INSIGHTS
--  TARGET_LAG = '60 minute'
--  WAREHOUSE = tc_wh --change to your own
--  AS

SELECT
  i.PROJECT_NAME,
  i.ASSIGNEE,
  i.CREATOR,
  AI_COMPLETE(
    model => 'claude-sonnet-4-5',
    prompt => 'You are an expert in project management and issue analysis. Given the following JIRA issue data, analyze the project patterns and provide structured insights. Only extract information found in the issue summaries. Be precise and factual in your analysis.' || 
              '\n\nJIRA ISSUE DATA:\n' || 
              SUMMARY || 
              '\nEND OF ISSUE DATA\n\n',
    response_format => {
      'type': 'json',
      'schema': {
        'type': 'object',
        'properties': {
          'PROJECT_ANALYSIS': {
            'type': 'object',
            'properties': {
              'part_component': {
                'type': 'string',
                'description': 'The name of the part that failed'
              },
              'shows_good_progress': {
                'type': 'boolean', 
                'description': 'TRUE if issues are being resolved in reasonable time with good outcomes'
              },
              'resolved': {
                'type': 'boolean',
                'description': 'TRUE if the issue appears to be resolved based on the comments'
              },
              'high_priority': {
                'type': 'boolean',
                'description': 'TRUE if issues indicate a high degree of critical importance'
              },
              'has_good_documentation': {
                'type': 'boolean',
                'description': 'TRUE if issue summaries are detailed and well-documented'
              },
              'project_summary': {
                'type': 'string',
                'description': 'Brief summary of overall project health and key patterns observed'
              }
            },
            'required': ['part_component', 'shows_good_progress', 'resolved', 'high_priority', 'has_good_documentation', 'project_summary'],
            'additionalProperties': false
          }
        },
        'required': ['PROJECT_ANALYSIS'],
        'additionalProperties': false
      }
    }
  ) AS project_analysis_json
FROM
  {{issues_table}} AS i
WHERE
  i.CREATED >= DATEADD(month, -6, CURRENT_DATE())
LIMIT 10;


In [ ]:
-- By expanding on these results, we're turning unstructured data into structured data! 
-- This could be particularly useful for quantafiable analysis for Cortex Agents / Snowflake Intelligence
-- Now you're able to build a reliable pipeline that goes:
-- raw data > AI transform > JSON extract

SELECT 
  PROJECT_NAME,
  ASSIGNEE,
  project_analysis_json:PROJECT_ANALYSIS.part_component::string as part_component,
  project_analysis_json:PROJECT_ANALYSIS.has_good_documentation::boolean as has_good_documentation,
  project_analysis_json:PROJECT_ANALYSIS.resolved::boolean as resolved,
  project_analysis_json:PROJECT_ANALYSIS.high_priority::boolean as high_priority,
  project_analysis_json:PROJECT_ANALYSIS.shows_good_progress::boolean as shows_good_progress,
  project_analysis_json:PROJECT_ANALYSIS.project_summary::string as project_summary
FROM {{Structured_outputs}};

# 📋 AI-Generated Maintenance Recommendations

One of the most valuable applications is generating actionable maintenance recommendations based on incident patterns and equipment history. Using the failure analysis and maintenance data, we can generate specific, contextual recommendations for maintenance teams.

This demonstrates how AI can augment human expertise while providing consistent, data-driven guidance.


# 🔄 Automated Issue Processing Pipeline

Now we'll create an automated pipeline using Dynamic Tables to process JIRA issues and extract key metadata. This pipeline continuously processes new issues as they arrive, extracting structured information using AI.

This approach mirrors modern data engineering practices where AI transforms unstructured data into structured, queryable formats.


In [ ]:
-- Create automated issue processing pipeline with Dynamic Table
-- This processes JIRA issues and extracts key metadata automatically as new issues arrive
-- Target lag controls how fresh the data is (adjust based on your needs)

CREATE OR REPLACE DYNAMIC TABLE JIRA_ISSUE_METADATA
  TARGET_LAG = '1 day'
  WAREHOUSE = tc_wh  -- Update to your warehouse
  AS
WITH issue_processing AS (
    SELECT 
        i.KEY as issue_key,
        i.SUMMARY as summary,
        i.CREATED as created_date,
        i.PRIORITY as priority,
        i.STATUS as status,
        i.PROJECT_NAME as project_name,
        i.ASSIGNEE as assignee,
        
        -- Extract manufacturing metadata using AI_COMPLETE with structured output
        AI_COMPLETE(
            model => 'claude-4-sonnet',
            prompt => 'You are an expert in manufacturing operations and issue analysis. Analyze this JIRA issue and extract key manufacturing metadata. Only extract information explicitly mentioned in the summary.' ||
                      '\n\nJIRA ISSUE:\n' || 
                      'Key: ' || i.KEY || '\n' ||
                      'Summary: ' || i.SUMMARY || '\n' ||
                      'Priority: ' || i.PRIORITY || '\n' ||
                      'Project: ' || i.PROJECT_NAME || '\n' ||
                      '\nEND OF ISSUE DATA\n\n',
            response_format => {
                'type': 'json',
                'schema': {
                    'type': 'object',
                    'properties': {
                        'equipment_name': {
                            'type': 'string',
                            'description': 'The specific equipment or asset mentioned (e.g., Press-12, Robot-Arm-7)'
                        },
                        'failure_type': {
                            'type': 'string',
                            'description': 'The type of failure or issue (e.g., Hydraulic Leak, Bearing Failure, Electrical Fault)'
                        },
                        'component_affected': {
                            'type': 'string',
                            'description': 'The specific component or part that failed (e.g., Valve-X102, Bearing-6204)'
                        },
                        'production_line': {
                            'type': 'string',
                            'description': 'The production line affected (e.g., LINE_1, LINE_2)'
                        },
                        'plant_location': {
                            'type': 'string',
                            'description': 'The plant or facility location mentioned (e.g., Plant A, Plant B)'
                        },
                        'downtime_minutes': {
                            'type': 'integer',
                            'description': 'Estimated downtime in minutes if mentioned'
                        },
                        'requires_parts': {
                            'type': 'boolean',
                            'description': 'TRUE if the issue indicates parts are needed for resolution'
                        },
                        'safety_concern': {
                            'type': 'boolean',
                            'description': 'TRUE if there are safety or compliance implications mentioned'
                        },
                        'issue_summary': {
                            'type': 'string',
                            'description': 'Brief 1-sentence summary of the core issue and impact'
                        }
                    },
                    'required': ['equipment_name', 'failure_type', 'component_affected', 'issue_summary'],
                    'additionalProperties': false
                }
            }
        ) as metadata_json,
        
        -- Classify urgency level
        AI_CLASSIFY(
            i.SUMMARY,
            [
                {'label': 'Critical', 'description': 'Production stopped, safety hazard, major equipment failure, immediate action required'},
                {'label': 'High', 'description': 'Significant production impact, equipment malfunction, urgent repair needed'},
                {'label': 'Medium', 'description': 'Moderate issue, scheduled maintenance, standard troubleshooting'},
                {'label': 'Low', 'description': 'Minor issue, routine maintenance, informational'}
            ],
            {'task_description': 'Classify manufacturing issue urgency', 'output_mode': 'single'}
        ) as urgency_classification,
        
        -- Classify department routing
        AI_CLASSIFY(
            i.SUMMARY,
            [
                {'label': 'Maintenance', 'description': 'Equipment repair, mechanical issues, preventive maintenance'},
                {'label': 'Electrical', 'description': 'Electrical faults, sensor issues, control systems'},
                {'label': 'Quality', 'description': 'Quality control, compliance, inspection issues'},
                {'label': 'Operations', 'description': 'Production efficiency, process optimization, line management'},
                {'label': 'Safety', 'description': 'Safety incidents, regulatory compliance, hazard reporting'}
            ],
            {'task_description': 'Route to appropriate department', 'output_mode': 'single'}
        ) as department_routing,
        
        -- Analyze sentiment/tone
        TRIM(AI_SENTIMENT(i.SUMMARY):categories[0].sentiment) as issue_sentiment
        
    FROM {{issues_table}} i
)
SELECT 
    issue_key as ISSUE_KEY,
    summary as ORIGINAL_SUMMARY,
    created_date as CREATED_DATE,
    priority as ORIGINAL_PRIORITY,
    status as STATUS,
    project_name as PROJECT_NAME,
    assignee as ASSIGNEE,
    
    -- Extract metadata from JSON response
    metadata_json:equipment_name::string as EQUIPMENT_NAME,
    metadata_json:failure_type::string as FAILURE_TYPE,
    metadata_json:component_affected::string as COMPONENT_AFFECTED,
    metadata_json:production_line::string as PRODUCTION_LINE,
    metadata_json:plant_location::string as PLANT_LOCATION,
    metadata_json:downtime_minutes::integer as DOWNTIME_MINUTES,
    metadata_json:requires_parts::boolean as REQUIRES_PARTS,
    metadata_json:safety_concern::boolean as SAFETY_CONCERN,
    metadata_json:issue_summary::string as AI_SUMMARY,
    
    -- Extract classification results
    urgency_classification:labels[0]::string as URGENCY_LEVEL,
    department_routing:labels[0]::string as DEPARTMENT,
    issue_sentiment as SENTIMENT,
    
    -- Derive additional insights
    CASE 
        WHEN lower(issue_sentiment) = 'negative' AND urgency_classification:labels[0]::string = 'Critical' THEN 'Immediate Action Required'
        WHEN metadata_json:safety_concern::boolean = TRUE THEN 'Safety Review Required'
        WHEN metadata_json:requires_parts::boolean = TRUE THEN 'Parts Procurement Needed'
        ELSE 'Standard Processing'
    END as ACTION_FLAG
    
FROM issue_processing;


In [ ]:
-- Query the processed metadata to see the AI-extracted insights
-- This shows how unstructured JIRA summaries are transformed into structured, queryable data

SELECT 
    ISSUE_KEY,
    EQUIPMENT_NAME,
    FAILURE_TYPE,
    COMPONENT_AFFECTED,
    PRODUCTION_LINE,
    PLANT_LOCATION,
    DOWNTIME_MINUTES,
    URGENCY_LEVEL,
    DEPARTMENT,
    ACTION_FLAG,
    AI_SUMMARY
FROM JIRA_ISSUE_METADATA
ORDER BY CREATED_DATE DESC
LIMIT 20;


In [ ]:
# JIRA Issue Recommendations Generator
st.header("📋 AI-Powered Issue Management Recommendations")

with st.form("recommendations_form"):
    st.markdown("### Generate project recommendations based on issue patterns")
    
    # Get projects with recent issues
    recent_projects = session.sql(f"""
        SELECT DISTINCT 
            i.PROJECT_NAME,
            COUNT(i.ID) as issue_count,
            MAX(i.CREATED) as last_issue
        FROM {issues_table} i
        WHERE i.CREATED >= DATEADD(month, -3, CURRENT_DATE())
        GROUP BY i.PROJECT_NAME
        ORDER BY issue_count DESC, last_issue DESC
        LIMIT 20
    """).to_pandas()

    col1, col2, col3 = st.columns(3)    
    
    selected_project_rec = col1.selectbox(
        "Select Project for Recommendations:",
        options=recent_projects['PROJECT_NAME'].tolist() if not recent_projects.empty else []
    )
    
    recommendation_type = col2.selectbox(
        "Recommendation Focus:",
        ["Process Improvement", "Team Efficiency", "Issue Prevention", "Quality Enhancement", "Workflow Optimization"]
    )
    
    urgency = col3.selectbox(
        "Priority Level:",
        ["High Priority", "Medium Priority", "Low Priority", "Routine"]
    )
    
    generate_recommendations = st.form_submit_button("Generate Recommendations")

if generate_recommendations and selected_project_rec:
    # Get issue patterns and context for the selected project
    recommendations_context = session.sql(f"""
        WITH recent_issues AS (
            SELECT 
                i.SUMMARY,
                i.CREATED,
                i.PRIORITY,
                i.STATUS,
                i.ASSIGNEE
            FROM {issues_table} i
            WHERE i.PROJECT_NAME = '{selected_project_rec}'
            ORDER BY i.CREATED DESC
            LIMIT 10
        ),
        project_context AS (
            SELECT 
                i.PROJECT_NAME,
                i.PROJECT_CODE,
                COUNT(i.ID) as total_issues,
                COUNT(CASE WHEN i.STATUS IN ('Open', 'In Progress', 'To Do') THEN 1 END) as open_issues
            FROM {issues_table} i
            WHERE i.PROJECT_NAME = '{selected_project_rec}'
            GROUP BY i.PROJECT_NAME, i.PROJECT_CODE
            LIMIT 1
        )
        SELECT 
            pc.PROJECT_NAME,
            pc.PROJECT_CODE,
            pc.total_issues,
            pc.open_issues,
            AI_COMPLETE(
                'claude-4-sonnet',
                CONCAT(
                    'Generate {recommendation_type} recommendations for project ', pc.PROJECT_NAME, ' (', pc.PROJECT_CODE, '). ',
                    'Priority level: {urgency}. Project has ', pc.total_issues, ' total issues with ', pc.open_issues, ' currently open. ',
                    'Based on these recent issues: ',
                    (SELECT LISTAGG(CONCAT(PRIORITY, ' - ', SUMMARY, ' (Assignee: ', ASSIGNEE, ', Status: ', STATUS, ')'), '; ') FROM recent_issues),
                    '. Provide specific, actionable recommendations including: 1) Process improvements 2) Team efficiency measures 3) Issue prevention strategies 4) Quality enhancements 5) Timeline and implementation steps. ',
                    'Format as a brief structured project improvement plan with clear priorities and actions. Return in markdown format. /n PROJECT PLAN:'
                )
            ) as project_recommendations
        FROM project_context pc
    """).collect()
    
    if recommendations_context:
        context = recommendations_context[0]
        
        st.success(f"Project Recommendations Generated for {context['PROJECT_NAME']}")
        
        # Display project info
        col1, col2, col3 = st.columns(3)
        with col1:
            st.markdown(f"**Project:** {context['PROJECT_NAME']}")
            st.markdown(f"**Code:** {context['PROJECT_CODE']}")
        with col2:
            st.markdown(f"**Total Issues:** {context['TOTAL_ISSUES']}")
            st.markdown(f"**Open Issues:** {context['OPEN_ISSUES']}")
        with col3:
            st.markdown(f"**Focus:** {recommendation_type}")
            st.markdown(f"**Priority:** {urgency}")
        
        # Display generated recommendations
        st.markdown("### 🔧 Generated Project Recommendations")
        
        recommendations_content = context['PROJECT_RECOMMENDATIONS']

        st.write(recommendations_content.strip('"').encode().decode('unicode_escape'))
        
        # Make recommendations editable
        #edited_recommendations = st.text_area(
        #    "Review and edit recommendations:",
        #    value=recommendations_content,
        #    height=400
        #)
        

# 🤖 Conversational Manufacturing Assistant

The ultimate goal is natural language interaction with our manufacturing data. Using Cortex Search, we can create a conversational assistant that answers complex operational questions, just like asking a senior manufacturing engineer.

This represents the future of manufacturing intelligence - instant, contextual, and conversational.


In [ ]:
-- Create Cortex Search service for JIRA data
-- This enables semantic search across all JIRA issues and project data

CREATE OR REPLACE CORTEX SEARCH SERVICE JIRA_SEARCH
  ON SUMMARY
  ATTRIBUTES CREATED, PROJECT_CODE, PRIORITY, STATUS
  WAREHOUSE = tc_wh
  TARGET_LAG = '1 day'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
  AS (
    SELECT
        i.KEY,
        i.ID,
        i.CREATED,
        i.PROJECT_CODE,
        i.PROJECT_NAME,
        i.SUMMARY,
        i.PRIORITY,
        i.STATUS,
        i.ASSIGNEE,
        i.CREATOR,
        i.REPORTER
    FROM {{issues_table}} i
);


In [ ]:
# Conversational JIRA Assistant
from snowflake.core import Root
from snowflake.cortex import complete
import json

st.header("🤖 Conversational JIRA Assistant")
st.markdown("Ask natural language questions about projects, issues, and team performance.")

# Sample questions to inspire users
st.markdown("""
**Try asking questions like:**
- "What are the most common issues in the Manufacturing Operations project?"
- "What are the common themes where an electrical fault happened?"
- "Which products repeatedly leak?
""")

root = Root(session)

# Conversational interface
with st.form("jira_query"):
    user_question = st.text_input(
        "Ask your JIRA question:",
        placeholder="What are the most common issues in the Manufacturing Operations project?"
    )
    
    search_submitted = st.form_submit_button("Search")

if search_submitted and user_question:
    try:
        # Search JIRA data
        search_service = (root
            .databases["ai_sol"]
            .schemas["jira"]
            .cortex_search_services["jira_search"]
        )
        
        search_results = search_service.search(
            query=user_question,
            columns=["KEY", "SUMMARY", "PROJECT_NAME", 
                    "PRIORITY", "CREATED", "ASSIGNEE", "STATUS"],
            limit=5
        )
        
        # Generate AI response based on search results
        model = 'claude-4-sonnet'
        
        context_prompt = f"""
        Based on the following JIRA issue data, answer this question: "{user_question}"
        
        JIRA Data:
        {search_results.to_str()}
        
        Provide a helpful, contextual answer that summarizes the relevant information and suggests project management actions if appropriate. 
        Be specific about issues, dates, and assignees mentioned. Focus on project insights and actionable recommendations.
        """
        
        ai_response = complete(model, context_prompt)
        
        st.markdown("### 🎯 AI Response")
        st.markdown(ai_response)
        
        st.markdown("### 📋 Relevant JIRA Issues")
        
        # Display search results in a more readable format
        results_json_string = search_results.to_json()  
        results_data = json.loads(results_json_string)  
        
        if results_data and 'results' in results_data:
            for i, result in enumerate(results_data['results'][:3]):  # Show top 3
                with st.expander(f"🎫 {result.get('KEY', 'Unknown Issue')} - {result.get('PRIORITY', 'Normal')} Priority"):
                    st.markdown(f"**Project:** {result.get('PROJECT_NAME', 'N/A')}")
                    st.markdown(f"**Created:** {result.get('CREATED', 'N/A')}")
                    st.markdown(f"**Assignee:** {result.get('ASSIGNEE', 'N/A')}")
                    st.markdown(f"**Status:** {result.get('STATUS', 'N/A')}")
                    st.markdown(f"**Summary:** {result.get('SUMMARY', 'No summary available')}")
        
    except Exception as e:
        st.error(f"Error: {str(e)}")
        st.info("Note: Make sure the JIRA_SEARCH service has been created and is available.")


# 📊 AI for Manufacturing Business Intelligence

## 🎯 Purpose
Learn how to leverage Snowflake's AI capabilities for advanced manufacturing intelligence through semantic analysis.

## 🔑 Key Components

- **Analyst Tool**: Enables quantitative analysis through natural language queries
- **Data Semantics**: Ensures accuracy and consistency in analysis through defined relationships and metrics
- **Semantic Views**: Can be created through:
  - UI-based configuration in Snowflake interface
  - SQL-based definition (demonstrated in next cell)

## 💡 Benefits
- Natural language querying of your manufacturing data
- Consistent metric definitions across your organization
- Enhanced data discoverability and understanding
- Improved data governance through semantic layer
- Real-time operational insights and predictive analytics


In [ ]:
CREATE
OR REPLACE SEMANTIC VIEW jira_analytics 
TABLES (
  issues AS AI_SOL.JIRA.ISSUES PRIMARY KEY (KEY) WITH SYNONYMS ('tickets', 'problems', 'tasks', 'bugs') COMMENT = 'JIRA issues and tickets information',
  metadata AS AI_SOL.JIRA.JIRA_ISSUE_METADATA PRIMARY KEY (ISSUE_KEY) WITH SYNONYMS ('issue metadata', 'extracted data', 'ai insights') COMMENT = 'AI-extracted metadata and insights from JIRA issues',
  projects AS AI_SOL.JIRA.PROJECTS PRIMARY KEY (PROJECT_CODE) WITH SYNONYMS ('project', 'initiatives') COMMENT = 'JIRA project configurations and details',
  users AS AI_SOL.JIRA.USERS PRIMARY KEY (DISPLAY_NAME) WITH SYNONYMS ('people', 'team members', 'assignees') COMMENT = 'JIRA users and team member information'
) 
RELATIONSHIPS (
  issues_to_metadata AS issues (KEY) REFERENCES metadata (ISSUE_KEY),
  issues_to_projects AS issues (PROJECT_CODE) REFERENCES projects (PROJECT_CODE),
  issues_to_assignee AS issues (ASSIGNEE) REFERENCES users (DISPLAY_NAME),
  issues_to_creator AS issues (CREATOR) REFERENCES users (DISPLAY_NAME),
  issues_to_reporter AS issues (REPORTER) REFERENCES users (DISPLAY_NAME)
) 
FACTS (
  issues.resolution_hours AS DATEDIFF (hour, issues.CREATED, issues.RESOLVED_TS),
  issues.issue_count AS 1 COMMENT = 'Count of individual issues',
  metadata.downtime_minutes AS metadata.DOWNTIME_MINUTES COMMENT = 'Production downtime in minutes'
) 
DIMENSIONS (
  issues.key AS KEY WITH SYNONYMS ('ticket key', 'issue number') COMMENT = 'Unique JIRA issue key',
  issues.summary AS SUMMARY WITH SYNONYMS ('title', 'description', 'issue summary') COMMENT = 'Brief description of the issue',
  issues.priority AS PRIORITY WITH SYNONYMS ('urgency', 'severity', 'importance') COMMENT = 'Priority level of the issue',
  issues.status AS STATUS WITH SYNONYMS ('state', 'condition', 'stage') COMMENT = 'Current status of the issue',
  issues.status_category AS STATUS_CATEGORY WITH SYNONYMS ('status group', 'category') COMMENT = 'Category grouping of status',
  issues.created AS CREATED COMMENT = 'Date when issue was created',
  issues.resolved_ts AS RESOLVED_TS COMMENT = 'Date when issue was resolved',
  issues.assignee AS ASSIGNEE WITH SYNONYMS ('assigned to', 'owner') COMMENT = 'Person assigned to the issue',
  issues.creator AS CREATOR WITH SYNONYMS ('created by', 'author') COMMENT = 'Person who created the issue',
  issues.reporter AS REPORTER WITH SYNONYMS ('reported by') COMMENT = 'Person who reported the issue',
  issues.channel AS CHANNEL WITH SYNONYMS ('source', 'origin') COMMENT = 'Channel through which issue was reported',
  issues.labels AS LABELS WITH SYNONYMS ('tags', 'categories') COMMENT = 'Labels or tags associated with the issue',
  metadata.equipment_name AS EQUIPMENT_NAME WITH SYNONYMS ('asset', 'machine', 'equipment') COMMENT = 'Equipment or asset involved in the issue',
  metadata.failure_type AS FAILURE_TYPE WITH SYNONYMS ('failure mode', 'issue type', 'problem type') COMMENT = 'Type of failure or malfunction',
  metadata.component_affected AS COMPONENT_AFFECTED WITH SYNONYMS ('part', 'component', 'failed part') COMMENT = 'Specific component that failed',
  metadata.production_line AS PRODUCTION_LINE WITH SYNONYMS ('line', 'manufacturing line') COMMENT = 'Production line where issue occurred',
  metadata.plant_location AS PLANT_LOCATION WITH SYNONYMS ('plant', 'facility', 'location') COMMENT = 'Plant or facility location',
  metadata.urgency_level AS URGENCY_LEVEL WITH SYNONYMS ('urgency', 'criticality') COMMENT = 'AI-classified urgency level',
  metadata.department AS DEPARTMENT WITH SYNONYMS ('responsible team', 'routing') COMMENT = 'Department responsible for resolution',
  metadata.sentiment AS SENTIMENT WITH SYNONYMS ('tone', 'sentiment analysis') COMMENT = 'Sentiment analysis of issue',
  metadata.action_flag AS ACTION_FLAG WITH SYNONYMS ('action required', 'flag', 'alert') COMMENT = 'Action flag for special handling',
  metadata.requires_parts AS REQUIRES_PARTS COMMENT = 'Whether parts are required for resolution',
  metadata.safety_concern AS SAFETY_CONCERN WITH SYNONYMS ('safety issue', 'compliance') COMMENT = 'Whether issue has safety implications',
  projects.project_code AS PROJECT_CODE WITH SYNONYMS ('project key', 'code') COMMENT = 'Unique project identifier',
  projects.project_name AS PROJECT_NAME WITH SYNONYMS ('project title', 'name') COMMENT = 'Name of the project',
  users.display_name AS DISPLAY_NAME WITH SYNONYMS ('name', 'person') COMMENT = 'Display name of the user',
  users.role AS ROLE WITH SYNONYMS ('position', 'job title') COMMENT = 'Role or position of the user',
  users.team AS TEAM WITH SYNONYMS ('department', 'group') COMMENT = 'Team or department of the user',
  users.location AS LOCATION WITH SYNONYMS ('office', 'site') COMMENT = 'Location of the user'
) 
METRICS (
  issues.total_issues AS COUNT(issues.KEY) COMMENT = 'Total number of issues',
  issues.resolved_issues AS COUNT(
    CASE
      WHEN issues.RESOLVED_TS IS NOT NULL THEN issues.KEY
      ELSE NULL
    END
  ) COMMENT = 'Number of resolved issues',
  issues.open_issues AS COUNT(
    CASE
      WHEN issues.STATUS IN ('Open', 'In Progress', 'To Do') THEN issues.KEY
      ELSE NULL
    END
  ) COMMENT = 'Number of open issues',
  issues.avg_resolution_time AS AVG(issues.resolution_hours) COMMENT = 'Average time to resolve issues in hours',
  issues.critical_issues AS COUNT(
    CASE
      WHEN issues.PRIORITY = 'Critical' THEN issues.KEY
      ELSE NULL
    END
  ) COMMENT = 'Number of critical priority issues',
  issues.high_priority_issues AS COUNT(
    CASE
      WHEN issues.PRIORITY IN ('Critical', 'High') THEN issues.KEY
      ELSE NULL
    END
  ) COMMENT = 'Number of high priority issues',
  metadata.total_downtime AS SUM(metadata.DOWNTIME_MINUTES) COMMENT = 'Total production downtime in minutes',
  metadata.avg_downtime AS AVG(metadata.DOWNTIME_MINUTES) COMMENT = 'Average downtime per issue in minutes',
  metadata.safety_issues AS COUNT(
    CASE
      WHEN metadata.SAFETY_CONCERN = TRUE THEN metadata.ISSUE_KEY
      ELSE NULL
    END
  ) COMMENT = 'Number of issues with safety concerns',
  metadata.parts_required_issues AS COUNT(
    CASE
      WHEN metadata.REQUIRES_PARTS = TRUE THEN metadata.ISSUE_KEY
      ELSE NULL
    END
  ) COMMENT = 'Number of issues requiring parts',
  metadata.critical_action_issues AS COUNT(
    CASE
      WHEN metadata.ACTION_FLAG = 'Immediate Action Required' THEN metadata.ISSUE_KEY
      ELSE NULL
    END
  ) COMMENT = 'Number of issues requiring immediate action',
  projects.total_projects AS COUNT(DISTINCT projects.PROJECT_CODE) COMMENT = 'Total number of projects',
  issues.issues_per_project AS COUNT(issues.KEY) / COUNT(DISTINCT issues.PROJECT_CODE) COMMENT = 'Average issues per project',
  users.total_users AS COUNT(DISTINCT users.DISPLAY_NAME) COMMENT = 'Total number of users',
  issues.issues_per_assignee AS COUNT(issues.KEY) / COUNT(DISTINCT issues.ASSIGNEE) COMMENT = 'Average issues per assignee',
  issues.resolution_rate AS (
    COUNT(
      CASE
        WHEN issues.RESOLVED_TS IS NOT NULL THEN issues.KEY
      END
    ) * 100.0 / COUNT(issues.KEY)
  ) COMMENT = 'Percentage of issues that have been resolved',
  issues.sla_compliance AS (
    COUNT(
      CASE
        WHEN issues.RESOLVED_TS IS NOT NULL
        AND DATEDIFF (hour, issues.CREATED, issues.RESOLVED_TS) <= 24 THEN issues.KEY
      END
    ) * 100.0 / COUNT(
      CASE
        WHEN issues.RESOLVED_TS IS NOT NULL THEN issues.KEY
      END
    )
  ) COMMENT = 'Percentage of issues resolved within 24 hours'
) COMMENT = 'Comprehensive semantic view for JIRA analytics with AI-extracted metadata for enhanced manufacturing intelligence';

In [ ]:
-- Semantic views can be used by AI features (Analyst & Intelligence)
-- or BI tools via SELECT *.. for example
-- This example shows project performance with AI-extracted manufacturing insights

SELECT * FROM SEMANTIC_VIEW(
    jira_analytics  
    DIMENSIONS projects.project_name, issues.priority, metadata.equipment_name, metadata.failure_type
    METRICS issues.total_issues, issues.resolved_issues, issues.avg_resolution_time, metadata.total_downtime, metadata.safety_issues
  );
  
-- Example: Analyze downtime by equipment and failure type
SELECT * FROM SEMANTIC_VIEW(
    jira_analytics
    DIMENSIONS metadata.equipment_name, metadata.failure_type, metadata.plant_location
    METRICS issues.total_issues, metadata.avg_downtime, metadata.total_downtime
  )
  WHERE metadata.equipment_name IS NOT NULL
  ORDER BY metadata.total_downtime DESC;


# 🚀 Next Steps: From Demo to Production

Congratulations! You've experienced the power of AI-driven manufacturing intelligence. Here's how to take this from demo to production:

## 1. 🎯 **Deploy Manufacturing Assistant**
Integrate the search service and AI capabilities into your CMMS or create a standalone Streamlit application for your maintenance teams.

## 2. 📊 **Expand Semantic Views**
Head to **AI > Studio > Cortex Analyst** to create and update semantic views of your manufacturing data, enabling natural language reporting and self-service analytics.

## 3. 🤖 **Build Manufacturing Agents**
Use the search services and AI functions as tools for intelligent agents that can:
- Automatically generate maintenance plans
- Monitor equipment health in real-time
- Predict failures before they occur
- Optimize maintenance schedules

## 4. 🔄 **Integrate with Your Systems**
Connect these AI capabilities directly to your CMMS, ERP, or existing manufacturing systems through APIs for seamless workflow integration.

## 5. 📈 **Scale Across Manufacturing Operations**
Roll out to your entire manufacturing organization with:
- Plant-specific insights
- Equipment expertise augmentation  
- Automated maintenance recommendations
- Real-time operational intelligence

---

### 💡 **Key Takeaways**

✅ **Conversational AI** transforms how maintenance teams interact with operational data

✅ **Automated Intelligence** surfaces insights that would take hours to find manually

✅ **Predictive Recommendations** help teams take proactive action before failures occur

✅ **Root Cause Awareness** keeps teams ahead of recurring problems

✅ **Maintenance Automation** ensures consistent processes across all equipment

---

*Ready to revolutionize your manufacturing intelligence? Start with your own JIRA data and see the transformation firsthand.*
